$$
H= H^{T*}
$$


$$
T^{-1} H T = H \qquad T=U_{T} K \qquad U_{T} U_{T}^* = \pm \mathcal{1}
$$

$$
C^{-1} H C= -H \qquad C=U_{C} K \qquad U_{C} U_{C}^* = \pm \mathcal{1}
$$

$$
S^{-1} H S = - H \qquad S= U_{S} \qquad U_{S} U_{S}^{\dagger} = \mathcal{1}
$$

Moving to Fourier Space

$$
T H(k) T^{-1} = H(-k)
$$
$$
C H(k) C^{-1} = - H(-k)
$$
$$
S H(k) S^{-1} = -H(k)
$$

Class | A | AIII | AI | BDI | D | DIII | AII | CII | C | CI
---   | --- | ---  | --- | --- | --- | ---  | --- | --- | --- | ---
T     | 0 | 0    | +  | +   | 0 | -    | -   | -   | 0 | +
C     | 0 | 0    | 0  | +   | + | +    | 0   | -   | - | -
S     | 0 | 0    | 0  | 1   | 0 | 1    | 0   | 1   | 0 | 1 

Eigenfact
$$
H = U \epsilon U^{-1} \qquad U^{-1} H U = \epsilon
$$
$$
U_{1}^{-1} H_{1} U_{1} = U_{2}^{-1} H_{2} U_{2}
$$
$$
U_{2} U_{1}^{-1} H_{1} U_{1} U_{2}^{-1}
= \left( U_{2} U_{1}^{-1} \right) H_{1} \left( U_{2} U_{1}^{-1} \right)^{-1} = H_{2}
$$

In [7]:
using LinearAlgebra

In [1]:
σx=[[0 1]
    [1 0]]
σy=[[0 -im]
    [im 0]]
σz=[[1 0]
    [0 -1]]

2×2 Array{Int64,2}:
 1   0
 0  -1

In [238]:
R1x(k::Array{Float64},v=1,w=2)=v-w*cos(k[1])
R1y(k::Array{Float64},w=2)=-w*sin(k[1])

R2x(k::Array{Float64})=-2*sin(k[1])
R2y(k::Array{Float64})=-2*sin(k[2])
R2z(k::Array{Float64},μ)=μ+2*sum(cos.(k))


Δ(k::Array{Float64};Δ0=1.0)=Δ0*im*σy*(k[1]*σx+k[2]*σy+k[3]*σz)
ξ(k::Array{Float64};m=1,μ=0.0)=sum(k.^2)/2m-μ

ξ (generic function with 1 method)

In [239]:
H1(k::Array{Float64},v=1,w=2)=R1x(k,v,w)*σx+R1y(k,w)*σy
H2(k::Array{Float64},t=1.0,μ=0.0,Δ0=1.0)=(t*cos(k[1])-μ)*σz-Δ0*sin(k[1])*σy
H3(k::Array{Float64},μ=1.0)=R2x(k)*σx+R2y(k)*σy+R2z(k,μ)*σz

H3 (generic function with 2 methods)

In [278]:
k=[.25,.656]
m=H1(k)
mp=-m

2×2 Array{Complex{Float64},2}:
      0.0+0.0im       0.937825-0.494808im
 0.937825+0.494808im       0.0+0.0im     

In [279]:
f=eigen(m)
fp=eigen(mp)

Eigen{Complex{Float64},Float64,Array{Complex{Float64},2},Array{Float64,1}}
eigenvalues:
2-element Array{Float64,1}:
 -1.0603538622353488
  1.0603538622353488
eigenvectors:
2×2 Array{Complex{Float64},2}:
  0.625397-0.329967im  0.625397-0.329967im
 -0.707107+0.0im       0.707107+0.0im     

In [280]:
U=(fp.vectors)*(f.vectors)^(-1)

2×2 Array{Complex{Float64},2}:
       0.0+0.0im       0.884445-0.466644im
 -0.884445-0.466644im       0.0+0.0im     

2×2 Array{Complex{Float64},2}:
 1.0+1.16355e-17im  0.0+0.0im        
 0.0+0.0im          1.0+1.12452e-17im

In [281]:
a=U*U
Up=U/sqrt(a)

2×2 Array{Complex{Float64},2}:
       0.0-0.0im       -0.466644-0.884445im
 -0.466644+0.884445im        0.0-0.0im     

In [286]:
kp=[.656,.732]
Up*H(kp)*Up^(-1)+H(kp)

2×2 Array{Complex{Float64},2}:
     -0.0+0.0im      0.752239+1.42574im
 0.752239-1.42574im       0.0-0.0im    

In [282]:
Up*Up

2×2 Array{Complex{Float64},2}:
 1.0+0.0im  -0.0+0.0im
 0.0+0.0im   1.0+0.0im

In [277]:
Up*conj(transpose(Up))

2×2 Array{Complex{Float64},2}:
 1.0+0.0im          0.0+1.11022e-16im
 0.0-1.11022e-16im  1.0+0.0im        

In [242]:
function Sf(H::Function,k::Array{Float64};atol=1e-10)
    M=H(k)
    Mp=-H(k)
    
    f=eigen(M)
    fp=eigen(Mp)
    
    U=(fp.vectors)*(f.vectors)^(-1)
    
    a=U*U
    Up=U/sqrt(a[1])
    
    return isapprox.(Up*M*Up^(-1),Mp,atol=atol),
        isapprox.(Up*Up,one(M),atol=atol)
end

Sf (generic function with 1 method)

In [245]:
function Cf(H::Function,k::Array{Float64};atol=1e-10)
    M=H(k)
    Mp=-H(-k)
    cM=conj.(M)
    
    f=eigen(cM)
    fp=eigen(Mp)
    
    U=(fp.vectors)*(f.vectors)^(-1)
    
    a=U*U
    Up=U/sqrt(a[1])
    
    return isapprox.(Up*cM*Up^(-1),Mp,atol=atol),
        isapprox.(Up*Up,one(M),atol=atol)
end

Cf (generic function with 1 method)

In [236]:
function Sa(M::Array{Float64};atol=1e-10)
    Mp=-M
    
    f=eigen(M)
    fp=eigen(Mp)
    
    U=(fp.vectors)*(f.vectors)^(-1)
    
    a=U*U
    Up=U/sqrt(a[1])
    
    return isapprox.(Up*M*Up^(-1),Mp,atol=atol),
        isapprox.(Up*Up,one(M),atol=atol)
end

Sa (generic function with 1 method)

In [ ]:
function Ca(M::Array{Float64};atol=1e-10)
    Mp=-H(-k)
    cM=conj.(M)
    
    f=eigen(cM)
    fp=eigen(Mp)
    
    U=(fp.vectors)*(f.vectors)^(-1)
    
    a=U*U
    Up=U/sqrt(a[1])
    
    return isapprox.(Up*cM*Up^(-1),Mp,atol=atol),
        isapprox.(Up*Up,one(M),atol=atol)
end